In [327]:
from PIL import Image
import glob,StringIO,codecs
from ToolsMoodleXML import *
import ToolsMoodleXML
import base64

In [328]:
def redimensionne(image):
    '''
    redimensionne l'image à la taille maximale autorisée (150,100)
    '''
    maxImageSize=(150.0,100.0)
    print image.size
    ratio=min(maxImageSize[0]/image.size[0],maxImageSize[1]/image.size[1])
    print ratio
    result=image.thumbnail(maxImageSize,Image.ANTIALIAS)
    tempImageSize=(image.size[0],image.size[1])
    print tempImageSize
    result=image.resize(tempImageSize,Image.ANTIALIAS)
    return result
#temp=Image.open(repertoire+"Images/16-K1-CHUNKS 461.png")
#redimensionne(temp)

In [329]:
def generate_montage(filenames):
    hMargin=10
    vMargin=10
    dragdropElements=[]
    maxImageSize=(150,100)
    images = [Image.open(filename) for filename in filenames]
    images = [image if (image.size[0]<=150) and (image.size[1]<=100) else redimensionne(image) for image in images]
    if None in images:
        print "problèmes avec %s"," ".join(filenames)
    widthMax = max(image.size[0] + hMargin for image in images)
    heightMax = max(image.size[1] + vMargin for image in images)
    row_size=600//(widthMax)
    height = heightMax*len(images)//row_size
    montage = Image.new(mode='RGBA', size=(widthMax*row_size, height), color=(0,0,0,0))

    max_x = 0
    max_y = 0
    offset_x = 0
    offset_y = 0
    for i,image in enumerate(images):
        blankImage=Image.new(mode='RGBA', size=(image.size[0], image.size[1]), color=(0, 0, 0, 0))
        montage.paste(blankImage, (offset_x, offset_y))
        imageIO=StringIO.StringIO()
        image.save(imageIO,"PNG")
        dragdropElements.append((base64.b64encode(imageIO.getvalue()),(offset_x, offset_y)))
        imageIO.close()
        max_x = max(max_x, offset_x + widthMax)
        max_y = max(max_y, offset_y + heightMax)

        if i % row_size == row_size-1:
            offset_y = max_y + vMargin
            offset_x = 0
        else:
            offset_x += hMargin + widthMax

    montage = montage.crop((0, 0, max_x, max_y+vMargin))
#    print (0, 0, max_x, max_y+vMargin)
    return (montage,dragdropElements)

In [330]:
repertoire="/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K1/"
nomImages=glob.glob(repertoire+"Images/16-K1-CHUNKS*.png")
nomImages=sorted(nomImages,key=lambda x: len(x))
nomCSV="Ordre.csv"

In [331]:
categorie=u"Kalaba16/K1/Ordre/Chunks/DDI/"
theme=u"Phonétique"
theme=u"Traductions"
#theme=u"Brut"

In [332]:
phrasesLignes=codecs.open(repertoire+nomCSV,"r",encoding="utf8").readlines()
phraseChunks=[]
for p in phrasesLignes:
    chunks=[c for c in p.strip().split("\t") if c!=""]
    phraseChunks.append(chunks)
#phraseChunks

In [333]:
phrasesLignes=codecs.open(repertoire+nomCSV.replace(".csv",".txt"),"r",encoding="utf8").readlines()
prononciations=[]
traductions=[]
for p in phrasesLignes:
    prononciation,traduction=p.strip().split(";")
    prononciations.append(prononciation.strip())
    traductions.append(traduction.strip())

In [334]:
def xmlExport(phraseChunks,numImage=0,numSlice=0):
    questions=XMLSeries(categorie+theme)
    for numPhrase,phrase in enumerate(phraseChunks):
        filenames=nomImages[numImage+numPhrase:numImage+numPhrase+len(phrase)]
        numImage+=len(phrase)
        (montage,hPositions)=generate_montage(filenames)
        imageIO=StringIO.StringIO()
        montage.save(imageIO,"PNG")
        base64Image=base64.b64encode(imageIO.getvalue())
        imageIO.close()
        consigne=u"Mettez les morceaux de la phrase suivante dans l'ordre<br>"
        if theme==u"Phonétique":
            consigne+=prononciations[numPhrase]+"<br>"
        elif theme==u"Traductions":
            consigne+=traductions[numPhrase]+"<br>"
        ddi=DragDropImage(u"Chunks%03d"%(numPhrase+numSlice*tailleSlice),
                         consigne,
                         base64Image,
                         hPositions,
                         drag="image"
                         )
        questions.addExercice(ddi)

    xmlOutput=MoodleXML()
    xmlOutput.addQuiz(questions.getSeries())

    xmlFichier=repertoire+nomCSV.replace(".csv","-%s-%02d.xml"%(theme,numSlice))
    with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
        output.write(xmlOutput.getXML())
    return (numImage)

In [335]:
def chunker(seq, size):
    return [seq[pos:pos + size] for pos in xrange(0, len(seq), size)]

In [336]:
numPhrase=0
tailleSlice=500
for numGroupe,groupe in enumerate(chunker(phraseChunks,tailleSlice)):
    print numGroupe,
    numPhrase=xmlExport(groupe,numPhrase,numGroupe)

0 (145, 103)
0.970873786408
(140, 100)
(145, 103)
0.970873786408
(140, 100)
